# Creating A Movie Barcode
김소희 (서강대학교 국제한국학과)
- - - 



## 1. What is a movie barcode?

'Movie Barcode(무비바코드)'란 영화의 각 프레임을 한 픽셀 너비로 만든 후 이어붙여 영화 전체를 바코드같이 생긴 하나의 이미지로 만든 것이다. 완성된 무비바코드를 살펴보면 영화의 전반적인 색감과 분위기를 알 수 있기 때문에, 일종의 color palette라고 볼 수도 있겠다. 다양한 무비바코드를 포스팅하는 [텀블러 계정](http://moviebarcode.tumblr.com/)이 있으니, 실제 예시를 보고 싶다면 참고하자.

## 2. Idea sketch: How to make a movie barcode with python

그렇다면, 파이썬으로 어떻게 무비바코드를 만들 수 있을까? 알고리즘을 간략히 아래의 네 단계로 요약할 수 있다.
- 프레임의 대표컬러값을 추출하는 함수를 정의한다. 
- 무비바코드를 만들고자 하는 비디오 파일을 오픈한다.
- 정의한 함수를 이용하여, 비디오 파일의 각 프레임의 대표컬러값을 추출한다.
- 각 프레임의 대표컬러값들을 이어붙여 무비바코드를 완성한다.
- 무비바코드를 이미지 파일로 저장한다.

이 때, 프레임의 대표컬러값을 정의하는 방법에는 여러가지가 있겠지만 이 리포트에서는 각 프레임의 정중앙에 위치한 좌표의 컬러값을 대표값으로 사용하고자 한다.

## 3. Practice: Creating a movie barcode of 'Little Forest(2018)'
위의 아이디어 스케치를 따라 영화 '리틀포레스트(2018)'의 무비바코드를 만들어보자. 편의상 [리틀포레스트의 예고편 영상](https://www.youtube.com/watch?v=IWpBGln6bNk)을 사용하였다.


#### 3.0. 필요한 패키지들을 import 한다



In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imageio

#### 3.1. 프레임의 대표컬러값을 추출하는 함수를 정의한다

In [ ]:
def getRepColor(img):
    rcenter = img.shape[0]//2
    ccenter = img.shape[1]//2
    color = img[rcenter, ccenter]
    return color

`img.shape` 함수는 이미지의 모양을 보여주는데, 순서대로 (height, width, depth)를 return한다. 따라서, `img.shape[0]`은 해당 이미지의 높이, `img.shape[1]`은 너비를 의미한다. `rcenter`와 `ccenter`에서 높이와 너비를 각각 반으로 나누어 y, x좌표로 넣으면 이미지의 정중앙 좌표를 가리키게 되는 것이다. </br> `color = img[rcenter, ccenter]`을 통해 정중앙 좌표의 컬러값을 추출하고, 이를 함숫값으로 반환한다. 앞에서 얘기한 대로 정중앙 좌표의 컬러값을 프레임의 대표값으로 사용할 것이기 때문이다. </br> </br> 이때 주의할 점은 openCV에서는 색을 b,r,g 순서로 읽는다는 것이다. 따라서 이를 r,g,b로 바꿔주는 별도의 작업이 필요한데, 이는 아래에서 다시 다루도록 하자.

#### 3.2. 무비바코드를 만들고자 하는 비디오 파일을 오픈한다

In [ ]:
filename = 'little_forest.mp4'
cap = cv2.VideoCapture(filename) 

`cv2.VideoCapture()` 함수를 사용하여 비디오 캡쳐 객체를 생성하고, 사용할 비디오 파일을 오픈한다

In [ ]:
CList = []

프레임들의 대표컬러값을 넣을 리스트를 미리 만들어 둔다.

#### 3.3. 정의한 함수를 이용하여, 비디오 파일의 각 프레임의 대표컬러값을 추출한다

In [ ]:
while(True): 
    ret, frame = cap.read()
    if ret == False:
        break 
    c = getRepColor(frame)
    CList.append(c)

cap.release()

`ret, frame = cap.read()`로 비디오의 한 프레임씩 읽는다. 제대로 프레임을 읽으면 ret 값이 True, 실패하면 False로 나타난다. 따라서, `While(True)`는 프레임을 읽는 동안 계속 반복된다는 뜻이고, `if ret == False: break`는 프레임을 끝까지 다 읽어 더 이상 읽는데 실패하면 while문을 끝내라는 의미이다. </br> 앞서 정의한 `getRepColor()`함수를 사용하여 각 프레임의 컬러값을 추출하고, 이를 `append`로 CList에 추가한다. 이제 CList에는 모든 프레임들의 컬러값이 들어있게 된다. 마지막으로 `cap.release()`로 오픈한 캡쳐 객체를 해제한다.

#### 3.4. 각 프레임의 대표컬러값들을 이어붙여 무비바코드를 완성한다.

In [ ]:
img = np.zeros((1000,len(CList),3)) 

`np.zeros()` 함수는 앞서 언급한 shape 함수와 같이 (height, width, depth)를 return 한다. `np.zeros((1000, len(CList), 3))`을 통해 높이가 1000, 너비가 CList의 길이, 채널이 3인 검은색 빈 공간 이미지를 만들 수 있다. 이 이미지 위에 무비바코드를 입힐 것이므로, 무비바코드의 이미지 크기를 생각하여 설정하여야 한다. </br> </br> 이 때, 너비가 `len(CList)`인 이유는 무비바코드의 정의를 생각해보면 알 수 있다. 무비바코드는 각 프레임을 한 픽셀 너비로 만든 후 이어붙인 것이기 때문에, 무비바코드의 너비는 프레임의 수와 동일하다. 앞서 각 프레임의 컬러값들을 CList에 추가하였으므로 CList의 길이는 프레임의 수와 같아지게 된다. (높이는 임의로 조정 가능하다.)

In [ ]:
for i in range(len(CList)):
    img[:,i] = CList[i] 

img에 무비바코드를 입히는 과정이다. `[:]`가 처음부터 끝까지(전체 범위)를 의미하므로, `img[:,i]`는 img의 y좌표 전체, x좌표가 i일 때를 나타낸다. 'x=1'과 같이 x축에 수직이고 y축에 평행한 직선을 생각하면 될 것이다.(바코드의 선 하나) 우변의 `CList[i]`는 CList의 i번째 원소로, i번째 프레임의 컬러값을 의미한다. </br> </br> i가 0일 때를 생각해보자. 좌변은 `img[:,0]`이므로 img의 x좌표 0과, 전체 y좌표를 뜻한다. 우변은 `CList[0]`이므로 첫번째 프레임의 rgb 컬러값이다. 따라서, 이는 첫번째 프레임의 컬러값을 img에 입히는 것이다. (다시 말해, 바코드의 첫번째 줄을 긋는 것!) i가 1일 때는 바코드의 두번째 줄, 2일 때는 세번째 줄, 이렇게 CList의 길이(즉, 프레임의 수)만큼 계속 바코드를 그어 나가므로 for문이 끝날 때에는 모든 프레임의 컬러값이 무비바코드에 입혀져있게 된다.

#### 3.5. 무비바코드를 이미지 파일로 저장한다
이제, 만들어진 무비바코드를 저장하면 된다. 하지만 저장에 앞서 해야 하는 작업들이 몇 가지 있다.

In [ ]:
b, g, r = cv2.split(img)   
img2 = cv2.merge([r,g,b]) 

앞에서도 언급했듯 openCV는 색을 b,g,r 순서로 읽기 때문에 이를 r,g,b로 바꿔주어야 한다. `cv2.split()` 함수를 사용하여 img 파일을 b,g,r로 분리하고, `cv2.merge()`함수로 b,r을 바꾸어 merge 한다. 구분을 위하여 r,g,b 순서로 바꿔준 img 파일을 img2라고 칭하였다.

In [ ]:
img2 = img2.astype(np.uint8) #changing datatype of img2 to unit8
plt.imshow(img2) #showing img2
plt.show()

imageio.imwrite('little_forest_movie_barcode.png', img2)

cv2.destroyAllWindows() #closing the image display window

`imageio.imwrite()` 함수를 사용하여 img2를 이미지 파일로 저장한다. 

## 4. The Result

> Movie Barcode of 'Little Forest(2018)'

![image](little_forest_movie_barcode.png)

위의 이미지가 완성된 '리틀포레스트'의 무비바코드이다. 1분 30초 정도의 예고편으로 만든 것이기 때문에 프레임 수는 2000여개 정도이다. (실제 영화로 무비바코드를 만들게 되면 가로 길이가 훨씬 길어질 것이다.)